### Complex scalilng with Gaussian basis sets

In [1]:
show_plots=False

In [2]:
import numpy as np
from scipy.linalg import eig, eigh, eigvals, eigvalsh
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Qt5Agg')
%matplotlib qt5
import pandas as pd

In [27]:
#
#  extend path by location of the dvr package
#
import sys
sys.path.append('../../Python_libs')
from captools import simple_traj_der
from GTO_basis import GBR
from jolanta import Jolanta_3D

In [4]:
amu_to_au=1822.888486192
au2cm=219474.63068
au2eV=27.211386027
Angs2Bohr=1.8897259886

In [5]:
#
# Jolanata parameters a, b, c:
#
#   CS-DVR:   
#      bound state:  -7.17051 eV
#      resonance (3.1729556 - 0.16085j) eV
#
jparam=(0.028, 1.0, 0.028)

* Create a valence set $[\alpha_0, \alpha_0/s, \alpha_0/s^2, ..., \alpha_N]$
* Diagonalize **H** to compare with $E_0^{DVR}$
* Add diffuse functions $[\alpha_N/s_{df}, ...]$
* Diagonalize **H** again

In [16]:
sets=['GTO_unc', 'GTO_DZ', 'GTO_TZ']
bas = sets[2]
nval=10
a0=17.0
s=2
ndf=4
sdf=1.5
if bas == 'GTO_unc':
    contract = (0,0)
elif bas == 'GTO_DZ':
    contract = (1,1)  # one contracted, one uncontracted function
elif bas == 'GTO_TZ':
    contract = (1,2)  # one contracted, two uncontracted function
else:
    print('No such basis.')

fname='Traj_' + bas + '.csv'
print(fname)

Traj_GTO_TZ.csv


### Valence set
Compare the bound state with DVR: $E_0 = -7.17051$ eV

In [26]:
alpha_val=[a0]
for i in range(nval-1):
    alpha_val.append(alpha_val[-1]/s)
Val = GBR(alpha_val, jparam, contract=contract, diffuse=(0,0))
S, T, V = Val.STV()
Es, cs = eigh(T+V, b=S)
print(f'E0 = {Es[0]*au2eV:.6f}   Emax = {Es[-1]*au2eV:.6f}')
Val.print_exp()

if show_plots:
    scale=10
    xmax=15
    xs=np.linspace(0.1,xmax,200)
    Vs=Jolanta_3D(xs, jparam)
    plt.cla()
    plt.plot(xs,Vs*au2eV, '-', color="blue")
    for i in range(len(Es)):
        ys=Val.eval_vector(cs[:,i], xs)
        plt.plot(xs,scale*ys**2+Es[i]*au2eV, '-')
    plt.ylim(-8,10)
    plt.show()

E0 = -7.170439   Emax = 7.526354
      alpha         r0=1/sqrt(alpha)     Norm
   1.70000000e+01    2.42535625e-01    1.0070e+02
   8.50000000e+00    3.42997170e-01    4.2341e+01
   4.25000000e+00    4.85071250e-01    1.7802e+01
   2.12500000e+00    6.85994341e-01    7.4848e+00
   1.06250000e+00    9.70142500e-01    3.1470e+00
   5.31250000e-01    1.37198868e+00    1.3231e+00
   2.65625000e-01    1.94028500e+00    5.5631e-01
   1.32812500e-01    2.74397736e+00    2.3390e-01
   6.64062500e-02    3.88057000e+00    9.8343e-02
   3.32031250e-02    5.48795472e+00    4.1348e-02


### Extend the basis by a diffuse set

In [18]:
Bas = GBR(alpha_val, jparam, contract=contract, diffuse=(ndf,sdf))
S, T, V = Bas.STV()
Es, cs = eigh(T+V, b=S)
nEs = len(Es)
print(f'E0 = {Es[0]*au2eV:.6f}   Emax = {Es[-1]*au2eV:.6f}')

if show_plots:
    Emax=10 # eV
    plt.cla()
    plt.plot(xs,Vs*au2eV, '-', color="blue")
    for i, E in enumerate(Es):
        ys=Bas.eval_vector(cs[:,i], xs)
        plt.plot(xs,scale*ys**2+E*au2eV, '-')
        if E*au2eV > Emax:
            break

    plt.ylim(-10,Emax+1)
    plt.show()

E0 = -7.170445   Emax = 12.806117


***
### CS
Example for testing library. With:

`nval=10, a0=17.0, s=2, ndf=4, sdf=1.5, theta = 10 deg`
1. `(-7.171756, 0.000583)`
2. `(0.327329, -0.171381)`
3. `(1.228884, -0.692745)`
4. `(3.165013, -0.147167)`
5. `(3.431676, -1.806220)`

In [19]:
if False:
    theta=10.0/180.0*np.pi
    print("theta = %f" % (theta))
    H_theta = Bas.H_theta(theta)
    energies = eigvals(H_theta, b=S)
    energies.sort()
    energies*=au2eV
    for e in energies:
        print("(%f, %f)" % (e.real, e.imag))

### $\theta$-run

In [20]:
n_keep=nEs

theta_min=0
theta_max=16
n_theta=101
thetas=np.linspace(theta_min, theta_max, num=n_theta)
run_data = np.zeros((n_theta,n_keep), complex)  # array used to collect all theta-run data

for i, tdeg in enumerate(thetas):
    theta = tdeg/180.0*np.pi
    H_theta = Bas.H_theta(theta)
    energies = au2eV * eigvals(H_theta, b=S)
    energies.sort()
    run_data[i,:] = energies[0:n_keep]
    print(i+1, end=" ")
    if (i+1)%10==0:
        print()

1 2 3 4 5 6 7 8 9 10 
11 12 13 14 15 16 17 18 19 20 
21 22 23 24 25 26 27 28 29 30 
31 32 33 34 35 36 37 38 39 40 
41 42 43 44 45 46 47 48 49 50 
51 52 53 54 55 56 57 58 59 60 
61 62 63 64 65 66 67 68 69 70 
71 72 73 74 75 76 77 78 79 80 
81 82 83 84 85 86 87 88 89 90 
91 92 93 94 95 96 97 98 99 100 
101 

Raw $\eta$ trajectories

In [21]:
plt.cla()
for i in range(0, n_keep):
    plt.plot(run_data[:,i].real,  run_data[:,i].imag, 'o')
plt.xlim(0,8)
plt.ylim(-2,0.5)
plt.show()

Get the resonance trajectory by naive nearest follow

In [22]:
#follow=3.7
follow=3.0
es=np.zeros(n_theta,complex)

for j in range(0,n_theta):
    i = np.argmin(abs(run_data[j,:]-follow))
    es[j] = run_data[j,i]
    follow = es[j]
plt.cla()
plt.ylim(-1,0.0)
plt.plot(es.real, es.imag, 'o-')
plt.show()

In [23]:
abs_der = np.abs(simple_traj_der(thetas, es))
plt.cla()
plt.plot(thetas, np.log10(abs_der), 'o-')
plt.xlabel(r'$\theta$ [deg.]')
plt.ylabel(r'$\log \vert dE/d \theta{} \vert $')
plt.show()

In [24]:
df = pd.DataFrame({"theta": thetas, 
                   "ReE":es.real, 
                   "ImE":es.imag,
                   "der": abs_der})
fname = 'Traj_' + bas + '.csv'
df.to_csv(fname, index=False)

## Use the box below or the Traj-analysis notebook

### DVR reference value
* (3.1729556212701966-0.16084889558781004j)
* derivative:   (-1.4354245561065682e-06+2.766522097519264e-08j)
* 1 deg change: (1.4358934654623567e-06+6.092757803832782e-08j)

In [25]:
j_opt = np.argmin(abs_der)
print(j_opt, thetas[j_opt])
Eres = es[j_opt]
print(Eres)
print(es[j_opt-1]-Eres)
print(es[j_opt+1]-Eres)
format='| %.6f  | %.6f | %.2f | %.1e |'
print(format % (Eres.real, Eres.imag, thetas[j_opt], abs_der[j_opt]))

29 4.64
(3.099076491793533-0.06820716073493643j)
(6.816480375926304e-05+0.0004275374599530396j)
(-0.00024872015692078975-0.0003456555770968983j)
| 3.099076  | -0.068207 | 4.64 | 2.6e-03 |
